# Practice Exercises

## Import Libraries

In [ ]:
import random

from langgraph.graph import START, END, StateGraph, MessagesState
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import ToolNode
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, ToolMessage
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from IPython.display import Image, display
from typing import Literal
import os
import re

from duckduckgo_search import DDGS

print("All imports successful")

All imports successful


In [2]:
load_dotenv()
api_key = os.getenv('paid_api')

if not api_key:
    raise ValueError("API key not found!")
print("API Key loaded successfully")

API Key loaded successfully


In [3]:
llm = ChatOpenAI(
    model = "gpt-40-mini",
    temperature = 0.7,
    api_key=api_key
)

print(f"LLM initialized {llm.model_name}")

LLM initialized gpt-40-mini


## Creating Custom Tools

In [ ]:

@ tool
def weather_tool(city: str) -> str:
    """
    Returns simulated weather for a given city.
    Use this tool when the weather of a given city is required.

    Args:
        city: The given city
    
    Returns:
        A simulated weather for the city
    """

    def simulate_weather(city: str) ->dict:
        """
        Generate deterministic fake weather data for a city.
        """
        random.seed(city.lower())

        conditions = ["Sunny", "Cloudy", "Rainy", "Thunderstorm", "Hazy"]
        
        return {
            "city": city.title(),
            "temperature_c": random.randint(22, 36),
            "condition": random.choice(conditions),
            "humidity_percent": random.randint(40, 90)
        }
    
    weather = simulate_weather(city)
    return (
        f"Weather in {weather['city']}:\n"
        f"- Temperature: {weather['temperature_c']}°C\n"
        f"- Condition: {weather['condition']}\n"
        f"- Humidity: {weather['humidity_percent']}%"
    )


In [ ]:
@tool
def dictionary(question: str) -> str:
    pass
    """
    Accepts a question and returns a definition from a simulated dictionary.

    Example inputs:
    - "What is the meaning of agent?"
    - "Define RAG"
    - "What does embedding mean?"
    """

    # Simulated dictionary
    dictionary = {
        "agent": "An entity that perceives its environment and acts upon it.",
        "tool": "A function or capability an agent can use to perform a task.",
        "rag": "Retrieval-Augmented Generation, combining retrieval with generation.",
        "llm": "Large Language Model trained on vast amounts of text data.",
        "embedding": "A numerical representation of text capturing semantic meaning."
    }

    # Normalize question
    word = question.lower().strip()


    # Lookup
    if word in dictionary:
        return f"{word.title()}: {dictionary[word]}"
    else:
        return f"{word.title()}: Definition not found in the simulated dictionary."


In [ ]:

@tool
def web_search(query: str) -> str:
    """
    Searches the web using DuckDuckGo and returns summarized results.

    Example inputs:
    - "Latest news on AI"
    - "What is LangGraph?"
    - "Weather patterns in Nigeria"
    """

    results_text = []

    with DDGS() as ddgs:
        results = ddgs.text(
            query,
            region="wt-wt",
            safesearch="moderate",
            max_results=5
        )

        for r in results:
            title = r.get("title", "No title")
            snippet = r.get("body", "No summary")
            source = r.get("href", "No link")

            results_text.append(
                f"Title: {title}\n"
                f"Summary: {snippet}\n"
                f"Source: {source}\n"
            )

    if not results_text:
        return "No results found."

    return "\n---\n".join(results_text)


## Binding tools to LLM

In [ ]:
tools= [weather_tool, dictionary, web_search]

llm_with_tools = llm.bind_tools(tools)
print(f"LLM bound to {len(tools)} tools")
print(f"Tools: {[tool.name for tool in tools]}")

## Implementing Conditional Routing